# Scientific Computing Report

This report will outline the code produced for this module as a library able to compute limit cycles of ODE's and PDE's as well as confirming the steady-states of diffusive PDE's

# A brief summary of your software (30% of the marks)

Structure of the modules,
How inputs are taken,
Checks,
Outputs

Start with the ODE solvers:

1. Present the difference between the solvers for solving a few steps of ODE of n dim
2. Isolate a periodic solution using shooting and know parameters
3. Trace out the parameter-period curve using both natural and pseudo-arclength continuation methods.


PDE solvers: 

Solver profiling and showing modularity
1. Explicit methods
2. implicit methods


# A description of the key software design decisions made (40% of the marks)

A run through of the code

# A reflective learning log (30% of the marks)